In [ ]:
import os
import pandas as pd
from pathlib import Path
from PIL import Image
import torch
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/soil-classification-part-2/model.pth'
TEST_DIR = Path('/content/drive/MyDrive/soil-classification-part-2/soil_competition-2025/test')
TEST_CSV = '/content/drive/MyDrive/soil-classification-part-2/soil_competition-2025/test_ids.csv'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32
NUM_CLASSES = 5  # Change if different

In [ ]:
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
class TestDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = Path(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        img_path = self.img_dir / img_name
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name



In [ ]:
test_dataset = TestDataset(TEST_CSV, TEST_DIR, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
predictions = []
image_names = []

In [ ]:
ith torch.no_grad():
    for images, names in test_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        image_names.extend(names)



In [ ]:

submission_df = pd.DataFrame({
    'image_id': image_names,
    'label': predictions
})
submission_path = '/content/drive/MyDrive/soil-classification-part-2/inference_predictions.csv'
submission_df.to_csv(submission_path, index=False)
print(f"✅ Predictions saved to: {submission_path}")